In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from utils import *
from transforms import *
from datasets import load_dataset, Dataset

In [475]:
df = init_transforms(meta=True)

In [4]:
datasets = [('glue', 'sst2'), 'ag_news']
tasks = df.task_name.unique().tolist()
trans = df.tran_type.unique().tolist()

tasks.reverse()
trans.reverse()

In [498]:
def one_hot_encode(y, nb_classes):
    if isinstance(y, np.ndarray):
        return y
    y = np.array(y)
    res = np.eye(nb_classes)[np.array(y).reshape(-1)]
    return res.reshape(list(y.shape)+[nb_classes])

def sample_Xy(text, label, num_sample=1):
    idx = np.random.randint(0, len(text), num_sample)
    return list(np.array(text)[idx]), list(np.array(label)[idx])

In [12]:
dataset = load_dataset('ag_news')['train']

text, label = dataset['text'], dataset['label'] 
new_text, new_label = [], []
    
text = np.array(text, dtype=np.string_)
label = pd.get_dummies(label).to_numpy(dtype=np.float)
    
batch_size= 1000

for t in [TextMix(), SentMix(), WordMix()]:

    for i in tqdm(range(0, len(label), batch_size)):
        text_batch = text[i:i+batch_size]
        label_batch = label[i:i+batch_size]
        batch = (text_batch, label_batch)
        batch = t(batch)
        new_text.extend(batch[0].tolist())
        new_label.extend(batch[1].tolist())

    save_dir = os.path.join('assets', 'AG_NEWS', t.__class__.__name__)
    npy_save(os.path.join(save_dir, 'text'), new_text)
    npy_save(os.path.join(save_dir, 'label'), new_label)

Using custom data configuration default
Reusing dataset ag_news (C:\Users\Fabrice\.cache\huggingface\datasets\ag_news\default\0.0.0\fb5c5e74a110037311ef5e904583ce9f8b9fbc1354290f97b4929f01b3f48b1a)
100%|████████████████████████████████████████████████████████████████████████████████| 120/120 [00:02<00:00, 50.65it/s]


In [534]:
dataset = load_dataset('glue', 'sst2')['train']
dataset.rename_column_('sentence', 'text')
task = 'sentiment'

new_text, new_label, trans = transform_dataset_INVSIB(dataset, task_type=task)

save_dir = os.path.join('assets', 'sst2', task, 'INVSIB')
npy_save(os.path.join(save_dir, 'text'), new_text)
npy_save(os.path.join(save_dir, 'label'), new_label)
npy_save(os.path.join(save_dir, 'trans'), trans)

Reusing dataset glue (C:\Users\Fabrice\.cache\huggingface\datasets\glue\sst2\1.0.0\7c99657241149a24692c402a5c3f34d4c9f1df5ac2e4c3759fadea38f6cb29c4)
100%|████████████████████████████████████████████████████████████████████████████| 67349/67349 [22:58<00:00, 48.86it/s]


In [119]:
dataset = load_dataset('ag_news')['train']
task = 'topic'

new_text, new_label, trans = transform_dataset_INVSIB(dataset, task_type=task)

save_dir = os.path.join('assets', 'ag_news', task, 'INVSIB')
npy_save(os.path.join(save_dir, 'text'), new_text)
npy_save(os.path.join(save_dir, 'label'), new_label)
npy_save(os.path.join(save_dir, 'trans'), trans)

Using custom data configuration default
Reusing dataset ag_news (C:\Users\Fabrice\.cache\huggingface\datasets\ag_news\default\0.0.0\fb5c5e74a110037311ef5e904583ce9f8b9fbc1354290f97b4929f01b3f48b1a)
120000it [5:50:16,  5.71it/s]


In [ ]:
n = 2 
for d in datasets:
    if type(d) == tuple and d[0] == 'glue':
        train, test = load_dataset(d[0], d[1], split=['train', 'test'])
        train.rename_column_('sentence', 'text')
        d = d[1]
    else:
        train, test = load_dataset(d, split=['train', 'test'])
    for task in tasks:
        task_df = df['task_name'] == task
        trans = df[task_df].tran_type.unique()
        for tran in trans:
            if (
                (d == 'ag_news' and task == 'topic'     and (tran == 'INV' or tran == 'SIB-mix'))  
             or (d == 'sst2'    and task == 'sentiment' and (tran == 'INV' or tran == 'SIB'))
            ):
                print('working on', d, task, tran)
                new_text, new_label, trans = transform_dataset(train, num_transforms=n, task=task, tran=tran)
                new_text = np.array(new_text)
                new_label = np.array(new_label)
                trans = np.array(trans, dtype=np.string_)
                save_dir = os.path.join('assets', d, task, tran)
                npy_save(os.path.join(save_dir, 'text3'), new_text)
                npy_save(os.path.join(save_dir, 'label3'), new_label)
                npy_save(os.path.join(save_dir, 'trans3'), trans)

In [ ]:
# n=2
# d = ('glue', 'sst2')
# task = 'sentiment'
# tran = None
# train, test = load_dataset(d[0], d[1], split=['train', 'test'])
# train.rename_column_('sentence', 'text')
# d = d[1]
# new_text, new_label, trans = transform_dataset(train, num_transforms=n, task=task, tran=tran)
# if tran is None:
#     tran = 'BOTH'
# new_text = np.array(new_text)
# new_label = np.array(new_label)
# trans = np.array(trans, dtype=np.string_)
# save_dir = os.path.join('assets', d, task, tran)
# npy_save(os.path.join(save_dir, 'text2'), new_text)
# npy_save(os.path.join(save_dir, 'label2'), new_label)
# npy_save(os.path.join(save_dir, 'trans2'), trans)

In [ ]:
# SST2_INV_text = npy_load("./assets/SST2/sentiment/INV/text2.npy")
# SST2_INV_label = npy_load("./assets/SST2/sentiment/INV/label2.npy")

# SST2_SIB_text = npy_load("./assets/SST2/sentiment/SIB/text2.npy")
# SST2_SIB_label = npy_load("./assets/SST2/sentiment/SIB/label2.npy")

In [ ]:
# INV_text = npy_load("./assets/AG_NEWS/topic/INV/text2.npy")
# INV_label = npy_load("./assets/AG_NEWS/topic/INV/label2.npy")

# SIB_text = npy_load("./assets/AG_NEWS/topic/SIB-mix/text2.npy")
# SIB_label = npy_load("./assets/AG_NEWS/topic/SIB-mix/label2.npy")